# Temporary Assistance for Needy Families (TANF)

TANF is a state-level benefit program run by the [Department of Health and Human Services](https://hhs.gov), part of which funds cash assistance.

From [benefits.gov](https://www.benefits.gov/benefit/613):

> The Temporary Assistance for Needy Families (TANF) program provides grant funds to states and territories to provide families with financial assistance and related support services. State-administered programs may include childcare assistance, job preparation, and work assistance.

## General formula

To calculate TANF entitlement, we use the general computation tree below (may vary depending on state and county):

- `tanf`: TANF entitlement
  - `tanf_amount_if_eligible`: amount if eligible
    - Definition: `tanf_max_amount - tanf_countable_income`
    - `tanf_max_amount`: maximum amount
      - **Parameters**: the maximum amount defined by different state TANF programs may vary due to these factors:
        - Household size: How many people live in the household.
        - "Region": Some states may define a "region"-level distinction for maximum amounts; for example, California (CalWORKS) defines two regions based on what county the household resides in.
        - Individual household properties such as caregiver or disability status.
        - Example: California (CalWORKS) has different maximum amounts based on these factors, which they define as "exempt/non-exempt".
        - State-specific policies.
    - `tanf_countable_income`: Amount deducted from TANF maximum amount based on income.
      - `tanf_gross_earned_income`: earned income.
        - **Parameter**: list of earned income sources summed.
      - `tanf_gross_unearned_income`: unearned income. (ADD DEF HERE)
        - **Parameter**: list of unearned income sources summed.
      - `deductions`: deductions from assessed income.
        - `earnings_deduction`: deduction amount based on earnings.
          - **Parameter**: percentage of earnings deducted from gross earned income.
          - **Parameter**: flat amount deducted from the household's gross earned income.
          - **Parameter**: flat amount deducted from each earner's gross earned income.
  - `is_tanf_eligible`: whether eligible for TANF
    - `is_tanf_enrolled`: whether a family is already enrolled in TANF.
    - `is_tanf_demographically_eligible`: demographic definition of TANF eligibility, which is mostly constant across the US.
      - Definition: If there are children (ages 0-17) present in the household, pregnant people, or there are people aged 18 years old that are currently enrolled in a school, the family is demographically eligible for TANF.
    - `is_tanf_economically_eligible`: Whether the family has sufficiently low income to qualify for eligibility.
      - `is_tanf_enrolled`: whether a family is already enrolled in TANF.
      - `is_tanf_continuous_eligible`:
        - `tanf_eligibility_income`: income measure used to assess eligibility for TANF.
          - **Parameters**: income definition varies depending on state policies:
            - **Parameter**: deductions from income per earner
            - **Parameter**: deductions from income per household
        - `tanf_max_amount` (defined above)
      - `is_tanf_initial_eligible`:
        - This variable is very similar to `is_continuous_eligible`, except for the initial employment deductions that are applied to determininig initial eligibility.
        - `tanf_eligibility_income`: income measure used to assess eligibility for TANF.
          - **Parameters**: income definition varies depending on state policies:
            - **Parameter**: deductions from income per earner
            - **Parameter**: deductions from income per household
        - `tanf_max_amount` (defined above)

  ## Examples

  A single parent of one in Illinois with earnings of \$250 per month will receive \$372.50 per month in TANF benefits.

In [1]:
from policyengine_us import IndividualSim
import pandas as pd
import plotly.express as px

sim_emp = IndividualSim(year=2022)
sim_emp.add_person(name="adult", age=30, employment_income=250 * 12)
sim_emp.add_person(name="child", age=10)
sim_emp.add_spm_unit(name="spm_unit", members=["adult", "child"])
sim_emp.add_household(
    name="household", members=["adult", "child"], state_code="IL"
)

print("TANF: ", sim_emp.calc("tanf") / 12)

TANF:  [372.5]


Their benefit falls steadily with earnings, until they earn \$430 per month, at which point they no longer qualify (assuming they are already enrolled).

In [2]:
LABELS = dict(
    employment_income="Monthly employment income",
    dividend_income="Monthly dividend income",
    monthly_income="Monthly income",
    income_source="Income source",
    monthly_tanf="Monthly TANF allotment",
    mtr="Marginal tax rate from TANF?",
    allotment="TANF allotment",
    state_code="State",
)


def make_df(state_code, enrolled, vary_var):
    sim = IndividualSim(year=2022)
    sim.add_person(name="adult", age=30)
    sim.add_person(name="child", age=10)
    sim.add_spm_unit(
        name="spm_unit", members=["adult", "child"], is_tanf_enrolled=enrolled
    )
    sim.add_household(
        name="household", members=["adult", "child"], state_code=state_code
    )

    sim.vary(vary_var, max=1500 * 12, step=120)

    return pd.DataFrame(
        dict(
            monthly_income=sim.calc(vary_var)[0] / 12,
            enrolled="Enrolled" if enrolled else "Not Enrolled",
            state_code=state_code,
            monthly_tanf=sim.calc("tanf")[0] / 12,
            vary_var=vary_var,
            # mtr=-sim.deriv("tanf", "employment_income"),
        )
    )


fig = px.line(
    make_df("IL", True, "employment_income"),
    "monthly_income",
    "monthly_tanf",
    labels=LABELS,
    title="TANF allotment for a two-person household in Illinois",
)

fig.update_layout(xaxis_tickformat="$,", yaxis_tickformat="$,")
fig.show()

This household's TANF benefit would vary depending on their state and whether they are already enrolled.

In [3]:
emp_df_combined = pd.concat(
    [
        make_df("IL", True, "employment_income"),
        make_df("IL", False, "employment_income"),
        make_df("CA", True, "employment_income"),
        make_df("CA", False, "employment_income"),
    ]
)

fig = px.line(
    emp_df_combined,
    "monthly_income",
    "monthly_tanf",
    color="enrolled",
    labels=LABELS,
    animation_frame="state_code",
    title="TANF allotment for a two-person household",
)

fig.update_layout(
    xaxis_tickformat="$,",
    yaxis_tickformat="$,",
    yaxis_range=[0, emp_df_combined.monthly_tanf.max() * 1.1],
    legend_title=None,
)
fig.show()

If the household's income is from Social Security instead of employment income, their TANF benefit phases out differently, and prior enrollment doesn't affect the benefit.

In [4]:
ss_df_combined = pd.concat(
    [
        make_df("IL", True, "social_security_disability"),
        make_df("IL", False, "social_security_disability"),
        make_df("CA", True, "social_security_disability"),
        make_df("CA", False, "social_security_disability"),
    ]
)

fig_ss = px.line(
    ss_df_combined,
    "monthly_income",
    "monthly_tanf",
    color="enrolled",
    labels=LABELS,
    animation_frame="state_code",
    title="TANF allotment for a two-person household as Social Security income varies",
)

fig_ss.update_layout(
    xaxis_tickformat="$,",
    yaxis_tickformat="$,",
    yaxis_range=[0, ss_df_combined.monthly_tanf.max() * 1.1],
    legend_title=None,
)
fig_ss.show()